In [85]:
import torch
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions

torch.manual_seed(42)

## 1. Convolutional Layer

### 1.1. Convolutional Operation

Input: $M \times N$

Kernel: $K \times O$

Output: $M - (K - 1) \times N - (O - 1)$

**Example 1**

In [86]:
input = torch.randint(5, (1, 6, 6), dtype=torch.float32) # 1x6x6: 1 batch, 6x6 grid, 5 possible values
input

tensor([[[2., 2., 1., 4., 1., 0.],
         [0., 4., 0., 3., 3., 4.],
         [0., 4., 1., 2., 0., 0.],
         [2., 1., 4., 1., 3., 1.],
         [4., 3., 1., 4., 2., 4.],
         [2., 0., 0., 4., 3., 4.]]])

In [87]:
# define convolutional layer
conv_layer = nn.Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=3, # create a kernel: 3 x 3
    bias=False # If True, adds a learnable bias to the output
)

In [88]:
conv_layer.weight # random weights of the kernel are updated during training (loss calculation, backpropagation,...)

Parameter containing:
tensor([[[[ 0.0520,  0.2693,  0.0364],
          [-0.1051,  0.0896, -0.0904],
          [ 0.1403,  0.2976,  0.1927]]]], requires_grad=True)

In [89]:
init_kernel_weight = torch.randint(
    high=2, # 0 or 1
    size=(conv_layer.weight.data.shape),
    dtype=torch.float32
)
init_kernel_weight

tensor([[[[1., 1., 0.],
          [1., 1., 0.],
          [1., 0., 1.]]]])

In [90]:
# init weight
conv_layer.weight.data = init_kernel_weight
conv_layer.weight

Parameter containing:
tensor([[[[1., 1., 0.],
          [1., 1., 0.],
          [1., 0., 1.]]]], requires_grad=True)

In [91]:
output = conv_layer(input)
output

tensor([[[ 9., 13.,  9., 13.],
         [14., 11., 13., 10.],
         [12., 17., 11., 14.],
         [12., 13., 13., 18.]]], grad_fn=<SqueezeBackward1>)

**Example 2**

In [92]:
input

tensor([[[2., 2., 1., 4., 1., 0.],
         [0., 4., 0., 3., 3., 4.],
         [0., 4., 1., 2., 0., 0.],
         [2., 1., 4., 1., 3., 1.],
         [4., 3., 1., 4., 2., 4.],
         [2., 0., 0., 4., 3., 4.]]])

In [93]:
# define convolutional layer
conv_layer = nn.Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=3, # create a kernel: 3 x 3
)

In [94]:
init_kernel_weight

tensor([[[[1., 1., 0.],
          [1., 1., 0.],
          [1., 0., 1.]]]])

In [95]:
# init weight
conv_layer.weight.data = init_kernel_weight
conv_layer.weight

Parameter containing:
tensor([[[[1., 1., 0.],
          [1., 1., 0.],
          [1., 0., 1.]]]], requires_grad=True)

In [96]:
conv_layer.bias # random bias

Parameter containing:
tensor([-0.1148], requires_grad=True)

In [97]:
# init bias
conv_layer.bias =  nn.Parameter(
    torch.tensor([1], dtype=torch.float32)
)
conv_layer.bias

Parameter containing:
tensor([1.], requires_grad=True)

In [98]:
output = conv_layer(input)
output

tensor([[[10., 14., 10., 14.],
         [15., 12., 14., 11.],
         [13., 18., 12., 15.],
         [13., 14., 14., 19.]]], grad_fn=<SqueezeBackward1>)

**Example 3**

In [99]:
input

tensor([[[2., 2., 1., 4., 1., 0.],
         [0., 4., 0., 3., 3., 4.],
         [0., 4., 1., 2., 0., 0.],
         [2., 1., 4., 1., 3., 1.],
         [4., 3., 1., 4., 2., 4.],
         [2., 0., 0., 4., 3., 4.]]])

In [100]:
# define convolutional layer
conv_layer = nn.Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=(2, 3), # create a kernel: 2 x 3
)

In [101]:
init_kernel_weight = torch.randint(
    high=2,
    size=(conv_layer.weight.data.shape),
    dtype=torch.float32
)
init_kernel_weight

tensor([[[[1., 0., 1.],
          [0., 1., 1.]]]])

In [102]:
# init weight & bias
conv_layer.weight.data = init_kernel_weight
conv_layer.weight

Parameter containing:
tensor([[[[1., 0., 1.],
          [0., 1., 1.]]]], requires_grad=True)

In [103]:
conv_layer.bias

Parameter containing:
tensor([0.3672], requires_grad=True)

In [104]:
# init bias
conv_layer.bias =  nn.Parameter(
    torch.tensor([1], dtype=torch.float32)
)
conv_layer.bias

Parameter containing:
tensor([1.], requires_grad=True)

In [105]:
output = conv_layer(input)
output

tensor([[[ 8., 10.,  9., 12.],
         [ 6., 11.,  6.,  8.],
         [ 7., 12.,  6.,  7.],
         [11.,  8., 14.,  9.],
         [ 6., 12., 11., 16.]]], grad_fn=<SqueezeBackward1>)

### 1.2. Padding

Padding is a technique used to preserve the spatial dimensions of the input volume. This is done by adding zeros around the border of the input volume.

Padding helps to control the size of the output volume.

Input: $M \times N$

Padding: $P \times Q$

New Input: $(M + 2P) \times (N + 2Q)$

Kernel: $K \times O$

Output: $(M + 2P - K - 1) \times (N + 2Q - O - 1)$

**Example 1**

In [106]:
input = torch.randint(5, (1, 4, 4), dtype=torch.float32)
input

tensor([[[2., 3., 1., 4.],
         [1., 1., 3., 2.],
         [0., 4., 3., 0.],
         [3., 2., 2., 0.]]])

In [107]:
init_kernel_weight = torch.randint(
    high=2,
    size=(conv_layer.weight.data.shape),
    dtype=torch.float32
)
init_kernel_weight

tensor([[[[1., 1., 1.],
          [1., 1., 1.]]]])

In [108]:
# define convolutional layer
conv_layer = nn.Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=3, # create a kernel: 3 x 3
    padding='same' # or 'valid'
    # 'same' helps to obtain the same size of the input and output
)

In [109]:
conv_layer.weight.data = init_kernel_weight
conv_layer.weight

Parameter containing:
tensor([[[[1., 1., 1.],
          [1., 1., 1.]]]], requires_grad=True)

In [110]:
# init bias
conv_layer.bias =  nn.Parameter(
    torch.tensor([1], dtype=torch.float32)
)
conv_layer.bias

Parameter containing:
tensor([1.], requires_grad=True)

In [111]:
output = conv_layer(input)
output

tensor([[[ 8., 12., 15., 11.],
         [ 7., 13., 14.,  9.],
         [10., 15., 12.,  6.],
         [ 6.,  8.,  5.,  3.]]], grad_fn=<SqueezeBackward1>)

**Example 2**

In [112]:
input

tensor([[[2., 3., 1., 4.],
         [1., 1., 3., 2.],
         [0., 4., 3., 0.],
         [3., 2., 2., 0.]]])

In [113]:
# define convolutional layer
conv_layer = nn.Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=3, # create a kernel: 3 x 3
    padding=(2, 1)
)

In [114]:
conv_layer.weight.data = init_kernel_weight
conv_layer.weight

Parameter containing:
tensor([[[[1., 1., 1.],
          [1., 1., 1.]]]], requires_grad=True)

In [115]:
# init bias
conv_layer.bias =  nn.Parameter(
    torch.tensor([1], dtype=torch.float32)
)
conv_layer.bias

Parameter containing:
tensor([1.], requires_grad=True)

In [116]:
output = conv_layer(input)
output

tensor([[[ 1.,  1.,  1.,  1.],
         [ 6.,  7.,  9.,  6.],
         [ 8., 12., 15., 11.],
         [ 7., 13., 14.,  9.],
         [10., 15., 12.,  6.],
         [ 6.,  8.,  5.,  3.],
         [ 1.,  1.,  1.,  1.]]], grad_fn=<SqueezeBackward1>)

### 1.3. Stride

Input: $M \times N$

Padding: $P \times Q$

New Input: $(M + 2P) \times (N + 2Q)$

Stride: $S \times T$

Kernel: $K \times O$

Output: $\left\lfloor \dfrac{M + 2P - K}{S} \right\rfloor \times \left\lfloor \dfrac{N + 2Q - O}{T} \right\rfloor$

**Example 1**

In [117]:
input = torch.randint(5, (1, 6, 6), dtype=torch.float32)
input

tensor([[[3., 4., 1., 2., 1., 0.],
         [4., 0., 2., 4., 1., 2.],
         [3., 1., 0., 1., 3., 1.],
         [3., 0., 1., 4., 0., 0.],
         [4., 0., 2., 0., 3., 3.],
         [2., 2., 2., 1., 3., 2.]]])

In [118]:
# define convolutional layer
conv_layer = nn.Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=3, # create a kernel: 3 x 3
    stride=2
)

In [119]:
conv_layer.weight.data = init_kernel_weight
conv_layer.weight

Parameter containing:
tensor([[[[1., 1., 1.],
          [1., 1., 1.]]]], requires_grad=True)

In [120]:
# init bias
conv_layer.bias =  nn.Parameter(
    torch.tensor([1], dtype=torch.float32)
)
conv_layer.bias

Parameter containing:
tensor([1.], requires_grad=True)

In [121]:
output = conv_layer(input)
output

tensor([[[15., 12.],
         [ 9., 10.],
         [13., 12.]]], grad_fn=<SqueezeBackward1>)

**Example 2**

In [122]:
input

tensor([[[3., 4., 1., 2., 1., 0.],
         [4., 0., 2., 4., 1., 2.],
         [3., 1., 0., 1., 3., 1.],
         [3., 0., 1., 4., 0., 0.],
         [4., 0., 2., 0., 3., 3.],
         [2., 2., 2., 1., 3., 2.]]])

In [123]:
# define convolutional layer
conv_layer = nn.Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=3, # create a kernel: 3 x 3
    stride=(1, 2)
)

In [124]:
conv_layer.weight.data = init_kernel_weight
conv_layer.weight

Parameter containing:
tensor([[[[1., 1., 1.],
          [1., 1., 1.]]]], requires_grad=True)

In [125]:
# init bias
conv_layer.bias =  nn.Parameter(
    torch.tensor([1], dtype=torch.float32)
)
conv_layer.bias

Parameter containing:
tensor([1.], requires_grad=True)

In [126]:
output = conv_layer(input)
output

tensor([[[15., 12.],
         [11., 12.],
         [ 9., 10.],
         [11., 11.],
         [13., 12.]]], grad_fn=<SqueezeBackward1>)

**Example 3**

In [127]:
input = torch.randint(5, (1, 4, 4), dtype=torch.float32)
input

tensor([[[4., 2., 1., 3.],
         [2., 3., 2., 1.],
         [0., 0., 1., 0.],
         [0., 1., 4., 2.]]])

In [128]:
# define convolutional layer
conv_layer = nn.Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=3, # create a kernel: 3 x 3
    padding=1,
    stride=(2, 2)
)

In [129]:
conv_layer.weight.data = init_kernel_weight
conv_layer.weight

Parameter containing:
tensor([[[[1., 1., 1.],
          [1., 1., 1.]]]], requires_grad=True)

In [130]:
# init bias
conv_layer.bias =  nn.Parameter(
    torch.tensor([1], dtype=torch.float32)
)
conv_layer.bias

Parameter containing:
tensor([1.], requires_grad=True)

In [131]:
output = conv_layer(input)
output

tensor([[[7., 7.],
         [6., 8.],
         [2., 8.]]], grad_fn=<SqueezeBackward1>)

**Example 4**

In [132]:
input = torch.randint(5, (1, 5, 7), dtype=torch.float32)
input

tensor([[[4., 4., 3., 4., 2., 0., 4.],
         [4., 1., 4., 0., 3., 2., 4.],
         [0., 1., 0., 2., 0., 3., 3.],
         [4., 1., 3., 1., 4., 1., 0.],
         [3., 0., 1., 2., 4., 4., 0.]]])

In [133]:
# define convolutional layer
conv_layer = nn.Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=3, # create a kernel: 3 x 3
    padding=1,
    stride=(2, 2)
)

In [134]:
output = conv_layer(input)
output

tensor([[[-1.1715, -2.5614, -1.9058, -0.5903],
         [-2.5259, -2.5943, -3.5760, -1.8482],
         [-1.1248, -1.4748, -2.2698, -1.4850]]], grad_fn=<SqueezeBackward1>)

## 2. Pooling Layer

### 2.1. Max Pooling Layer

**Example 1**

In [135]:
input = torch.randint(5, (1, 6, 6), dtype=torch.float32)
input

tensor([[[3., 4., 2., 3., 3., 0.],
         [0., 2., 4., 3., 4., 3.],
         [1., 2., 0., 3., 4., 2.],
         [2., 1., 3., 2., 0., 3.],
         [2., 3., 2., 2., 2., 0.],
         [4., 2., 2., 3., 2., 4.]]])

In [136]:
max_pool_layer = nn.MaxPool2d(kernel_size=2)

In [137]:
output = max_pool_layer(input)
output

tensor([[[4., 4., 4.],
         [2., 3., 4.],
         [4., 3., 4.]]])

**Example 2**

In [138]:
input

tensor([[[3., 4., 2., 3., 3., 0.],
         [0., 2., 4., 3., 4., 3.],
         [1., 2., 0., 3., 4., 2.],
         [2., 1., 3., 2., 0., 3.],
         [2., 3., 2., 2., 2., 0.],
         [4., 2., 2., 3., 2., 4.]]])

In [139]:
max_pool_layer = nn.MaxPool2d(
    kernel_size=2,
    stride=(1, 2)
)

In [140]:
output = max_pool_layer(input)
output

tensor([[[4., 4., 4.],
         [2., 4., 4.],
         [2., 3., 4.],
         [3., 3., 3.],
         [4., 3., 4.]]])

**Example 3**

In [141]:
input

tensor([[[3., 4., 2., 3., 3., 0.],
         [0., 2., 4., 3., 4., 3.],
         [1., 2., 0., 3., 4., 2.],
         [2., 1., 3., 2., 0., 3.],
         [2., 3., 2., 2., 2., 0.],
         [4., 2., 2., 3., 2., 4.]]])

In [142]:
max_pool_layer = nn.MaxPool2d(
    kernel_size=(2, 3),
    stride=(2, 2),
    padding=1
)

In [143]:
output = max_pool_layer(input)
output

tensor([[[4., 4., 3.],
         [2., 4., 4.],
         [3., 3., 3.],
         [4., 3., 4.]]])

**Example 4: MaxPool1d**

In [144]:
input

tensor([[[3., 4., 2., 3., 3., 0.],
         [0., 2., 4., 3., 4., 3.],
         [1., 2., 0., 3., 4., 2.],
         [2., 1., 3., 2., 0., 3.],
         [2., 3., 2., 2., 2., 0.],
         [4., 2., 2., 3., 2., 4.]]])

In [145]:
max_pool_layer = nn.MaxPool1d(
    kernel_size=3,
    stride=3
)

In [146]:
max_pool_layer(input)

tensor([[[4., 3.],
         [4., 4.],
         [2., 4.],
         [3., 3.],
         [3., 2.],
         [4., 4.]]])

### 2.2. Average Pooling Layer

**Example 1**

In [147]:
input

tensor([[[3., 4., 2., 3., 3., 0.],
         [0., 2., 4., 3., 4., 3.],
         [1., 2., 0., 3., 4., 2.],
         [2., 1., 3., 2., 0., 3.],
         [2., 3., 2., 2., 2., 0.],
         [4., 2., 2., 3., 2., 4.]]])

In [148]:
avg_pool_layer = nn.AvgPool2d(
    kernel_size=(3, 2),
    stride=(2, 2)
)

In [149]:
output = avg_pool_layer(input)
output

tensor([[[2.0000, 2.5000, 2.6667],
         [1.8333, 2.0000, 1.8333]]])

**Example 2**

In [150]:
input

tensor([[[3., 4., 2., 3., 3., 0.],
         [0., 2., 4., 3., 4., 3.],
         [1., 2., 0., 3., 4., 2.],
         [2., 1., 3., 2., 0., 3.],
         [2., 3., 2., 2., 2., 0.],
         [4., 2., 2., 3., 2., 4.]]])

In [151]:
avg_pool_layer = nn.AvgPool1d(
    kernel_size=3,
    stride=3
)

In [152]:
output = avg_pool_layer(input)
output

tensor([[[3.0000, 2.0000],
         [2.0000, 3.3333],
         [1.0000, 3.0000],
         [2.0000, 1.6667],
         [2.3333, 1.3333],
         [2.6667, 3.0000]]])

## 3. Flatten

In [153]:
input = torch.randint(5, (1, 3, 2), dtype=torch.float32)
input

tensor([[[3., 0.],
         [4., 2.],
         [2., 1.]]])

In [154]:
flatten_layer = nn.Flatten()

In [155]:
output = flatten_layer(input)
output

tensor([[3., 0., 4., 2., 2., 1.]])

# Excercise

### Ex01

In [156]:
# input
input = torch.tensor([[[2., 2., 1., 4., 1., 0.],
                       [0., 4., 0., 3., 3., 4.],
                       [0., 4., 1., 2., 0., 0.],
                       [2., 1., 4., 1., 3., 1.]]])

# define convolutional layer
conv_layer = nn.Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=3, # create a kernel: 3 x 3
    bias=False
)

In [157]:
init_kernel_weight = torch.tensor([[[[1., 1., 0.],
                                     [1., 0., 0.],
                                     [0., 0., 0.]]]])

init_kernel_weight

tensor([[[[1., 1., 0.],
          [1., 0., 0.],
          [0., 0., 0.]]]])

In [158]:
# init weight
conv_layer.weight.data = init_kernel_weight
conv_layer.weight

Parameter containing:
tensor([[[[1., 1., 0.],
          [1., 0., 0.],
          [0., 0., 0.]]]], requires_grad=True)

In [159]:
output = conv_layer(input)
output

tensor([[[4., 7., 5., 8.],
         [4., 8., 4., 8.]]], grad_fn=<SqueezeBackward1>)

In [160]:
# init bias
conv_layer.bias =  nn.Parameter(
    torch.tensor([2], dtype=torch.float32)
)
conv_layer.bias

Parameter containing:
tensor([2.], requires_grad=True)

In [161]:
output = conv_layer(input)
output


tensor([[[ 6.,  9.,  7., 10.],
         [ 6., 10.,  6., 10.]]], grad_fn=<SqueezeBackward1>)

### Ex02

In [162]:
# input
input = torch.tensor([[[2., 4., 2.],
                       [3., 3., 4.],
                       [3., 2., 0.],
                       [4., 0., 4.],
                       [1., 4., 0.]]])

# define convolutional layer
conv_layer = nn.Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=3, # create a kernel: 3 x 3
    bias=False,
    padding=1
)

In [163]:
input

tensor([[[2., 4., 2.],
         [3., 3., 4.],
         [3., 2., 0.],
         [4., 0., 4.],
         [1., 4., 0.]]])

In [164]:
init_kernel_weight = torch.tensor([[[[1., 0., 1.],
                                     [1., 1., 1.],
                                     [0., 1., 0.]]]])

init_kernel_weight

tensor([[[[1., 0., 1.],
          [1., 1., 1.],
          [0., 1., 0.]]]])

In [165]:
# init weight
conv_layer.weight.data = init_kernel_weight
conv_layer.weight

Parameter containing:
tensor([[[[1., 0., 1.],
          [1., 1., 1.],
          [0., 1., 0.]]]], requires_grad=True)

In [166]:
# init bias
conv_layer.bias =  nn.Parameter(
    torch.tensor([2], dtype=torch.float32)
)
conv_layer.bias

Parameter containing:
tensor([2.], requires_grad=True)

In [167]:
output = conv_layer(input)
output

tensor([[[11., 13., 12.],
         [15., 18., 13.],
         [14., 14., 11.],
         [ 9., 17.,  8.],
         [ 7., 15.,  6.]]], grad_fn=<SqueezeBackward1>)

In [168]:
conv_layer.stride = (2, 2)

In [169]:
output = conv_layer(input)
output

tensor([[[11., 12.],
         [14., 11.],
         [ 7.,  6.]]], grad_fn=<SqueezeBackward1>)

### Ex03 - Conv + Pooling

In [170]:
# input
input = torch.tensor([[[2., 4., 2.],
                       [1., 3., 2.],
                       [3., 2., 1.],
                       [0., 0., 1.],
                       [0., 0., 1.]]])
# define convolutional layer
conv_layer = nn.Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=(3,2), # create a kernel: 3 x 3
    bias=False,
)

In [171]:
input

tensor([[[2., 4., 2.],
         [1., 3., 2.],
         [3., 2., 1.],
         [0., 0., 1.],
         [0., 0., 1.]]])

In [172]:
init_kernel_weight = torch.tensor([[[[1., 1.],
                                     [1., 0.],
                                     [0., 0.]]]])

init_kernel_weight

tensor([[[[1., 1.],
          [1., 0.],
          [0., 0.]]]])

In [173]:
# init weight
conv_layer.weight.data = init_kernel_weight
conv_layer.weight

Parameter containing:
tensor([[[[1., 1.],
          [1., 0.],
          [0., 0.]]]], requires_grad=True)

In [174]:
# init bias
conv_layer.bias =  nn.Parameter(
    torch.tensor([1], dtype=torch.float32)
)
conv_layer.bias

Parameter containing:
tensor([1.], requires_grad=True)

In [175]:
max_pool_layer = nn.MaxPool2d(kernel_size=(1, 2))
avg_pool_layer = nn.AvgPool2d(kernel_size=(1, 2))

output = conv_layer(input)
output

tensor([[[ 8., 10.],
         [ 8.,  8.],
         [ 6.,  4.]]], grad_fn=<SqueezeBackward1>)

In [176]:
max_pool_layer(output)

tensor([[[10.],
         [ 8.],
         [ 6.]]], grad_fn=<MaxPool2DWithIndicesBackward0>)

In [177]:
avg_pool_layer(output)

tensor([[[9.],
         [8.],
         [5.]]], grad_fn=<AvgPool2DBackward0>)

### Ex04 - Convolutions on Gray Scale Images

In [178]:
# define convolutional layer
conv_layer = nn.Conv2d(
    in_channels=1,
    out_channels=1,
    kernel_size=(3,3), # create a kernel: 3 x 3
    bias=False,
)

In [179]:
!gdown 12Q7SpIVm4W2hT6QsiEuARLl5xXKMK8pE

/bin/bash: line 1: gdown: command not found


In [ ]:
import pandas as pd
import numpy as np
import cv2

test_df = pd.read_csv("test.csv")

In [ ]:
img = np.array(test_df.iloc[1].values).astype(np.uint8).reshape((28, 28))
img.shape

(28, 28)

In [ ]:
resized_img =  cv2.resize(img, (7, 7))
resized_img.shape

(7, 7)

In [ ]:
resized_img

array([[  0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,  43,  43,   0,   0],
       [  0,  30, 250, 230, 125, 251,   0],
       [  0, 191,  38,   0,   0,  81,   0],
       [  0, 241,   0,  35, 119, 250,   0],
       [  0,  49, 193, 198,  83,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0]], dtype=uint8)

In [ ]:
tensor_img = torch.tensor(resized_img).resize(1, 1, 7 , 7).float()

/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:836: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [ ]:
kernel_weight = nn.Parameter(torch.tensor([[[[1., 0. , 1.],
                                            [1., 0., -1.],
                                            [1., 0., -1.]]]]))
conv_layer.weight = kernel_weight
conv_layer.weight

Parameter containing:
tensor([[[[ 1.,  0.,  1.],
          [ 1.,  0., -1.],
          [ 1.,  0., -1.]]]], requires_grad=True)

In [ ]:
tensor_img

tensor([[[[  0.,   0.,   0.,   0.,   0.,   0.,   0.],
          [  0.,   0.,   0.,  43.,  43.,   0.,   0.],
          [  0.,  30., 250., 230., 125., 251.,   0.],
          [  0., 191.,  38.,   0.,   0.,  81.,   0.],
          [  0., 241.,   0.,  35., 119., 250.,   0.],
          [  0.,  49., 193., 198.,  83.,   0.,   0.],
          [  0.,   0.,   0.,   0.,   0.,   0.,   0.]]]])

In [ ]:
max_pool_layer = nn.MaxPool2d(kernel_size=(2, 2))

output = max_pool_layer(tensor_img)
output

tensor([[[[  0.,  43.,  43.],
          [191., 250., 251.],
          [241., 198., 250.]]]])

In [ ]:
output = conv_layer(tensor_img)
output

tensor([[[[-250., -243.,   82.,   22.,  168.],
          [-288.,   34.,  206.,  -59.,  168.],
          [ 212.,  657.,  294.,  185.,  244.],
          [-155.,  248.,   29.,   64.,  202.],
          [-193.,  127.,  229.,  483.,  202.]]]],
       grad_fn=<ConvolutionBackward0>)

In [ ]:
max_pool_layer = nn.MaxPool2d(kernel_size=(2, 2))

max_pool_layer(output)

tensor([[[[ 34., 206.],
          [657., 294.]]]], grad_fn=<MaxPool2DWithIndicesBackward0>)